In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
from sklearn.pipeline import Pipeline
##feature Scaling
from sklearn.preprocessing import StandardScaler

In [3]:
classifier = pkl.load(open('rf_clasf.pkl', 'rb'))
regressor = pkl.load(open('rf_reg.pkl', 'rb'))

In [66]:
type(classifier)



sklearn.ensemble._forest.RandomForestClassifier

In [34]:
df=pd.read_csv("Cobined_Flight_Weather.csv",index_col=[0])

In [35]:
df

,FlightDate,Quarter,Year,Month,Origin,Dest,DayofMonth,DepTime,DepDel15,CRSDepTime,...,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,airport_code
0,2016-01-01,1,2016,1,SEA,JFK,1,741.0,0.0,745,...,1030,0,23,8,34,29,66,2016-01-01,700,SEA
1,2016-01-02,1,2016,1,SEA,JFK,2,737.0,0.0,745,...,1024,0,24,8,36,32,64,2016-01-02,700,SEA
2,2016-01-03,1,2016,1,SEA,JFK,3,743.0,0.0,745,...,1018,0,20,7,32,28,62,2016-01-03,700,SEA
3,2016-01-04,1,2016,1,SEA,JFK,4,737.0,0.0,745,...,1010,97,32,8,34,31,95,2016-01-04,700,SEA
4,2016-01-05,1,2016,1,SEA,JFK,5,708.0,0.0,710,...,1001,100,35,10,38,35,91,2016-01-05,700,SEA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840269,2017-09-25,3,2017,9,SFO,JFK,25,2038.0,0.0,2045,...,1011,8,61,20,71,71,71,2017-09-25,2000,SFO
1840270,2017-09-25,3,2017,9,EWR,MCO,25,1200.0,0.0,1205,...,1017,8,68,8,83,83,60,2017-09-25,1200,EWR
1840271,2017-09-25,3,2017,9,MCO,EWR,25,1143.0,0.0,1141,...,1012,19,74,10,82,82,77,2017-09-25,1100,MCO
1840272,2017-09-25,3,2017,9,LAS,JFK,25,2157.0,0.0,2159,...,1011,1,27,19,64,64,24,2017-09-25,2200,LAS


In [36]:
df.columns

Index(['FlightDate', 'Quarter', 'Year', 'Month', 'Origin', 'Dest',
       'DayofMonth', 'DepTime', 'DepDel15', 'CRSDepTime', 'DepDelayMinutes',
       'OriginAirportID', 'DestAirportID', 'ArrTime', 'CRSArrTime', 'ArrDel15',
       'ArrDelayMinutes', 'rounded_time', 'windspeedKmph', 'winddirDegree',
       'weatherCode', 'precipMM', 'visibility', 'pressure', 'cloudcover',
       'DewPointF', 'WindGustKmph', 'tempF', 'WindChillF', 'humidity', 'date',
       'time', 'airport_code'],
      dtype='object')

In [37]:
df = df.drop(columns = ['airport_code','rounded_time','date','FlightDate','OriginAirportID', 'DestAirportID','WindChillF', 'CRSArrTime', 'ArrTime', 'time', 'Quarter'])

In [38]:
df.columns

Index(['Year', 'Month', 'Origin', 'Dest', 'DayofMonth', 'DepTime', 'DepDel15',
       'CRSDepTime', 'DepDelayMinutes', 'ArrDel15', 'ArrDelayMinutes',
       'windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM',
       'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph',
       'tempF', 'humidity'],
      dtype='object')

In [39]:
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
df["Origin"] = labelEncoder.fit_transform(df["Origin"])
df["Dest"] = labelEncoder.fit_transform(df["Dest"])

In [40]:
X = df.drop(columns = ['ArrDel15', 'ArrDelayMinutes'])

In [41]:
X

,Year,Month,Origin,Dest,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,humidity
0,2016,1,13,6,1,741.0,0.0,745,0.0,7,107,113,0.0,10,1030,0,23,8,34,66
1,2016,1,13,6,2,737.0,0.0,745,0.0,7,119,113,0.0,10,1024,0,24,8,36,64
2,2016,1,13,6,3,743.0,0.0,745,0.0,6,85,113,0.0,10,1018,0,20,7,32,62
3,2016,1,13,6,4,737.0,0.0,745,0.0,5,9,143,0.2,3,1010,97,32,8,34,95
4,2016,1,13,6,5,708.0,0.0,710,0.0,5,62,266,0.1,6,1001,100,35,10,38,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840269,2017,9,14,6,25,2038.0,0.0,2045,0.0,18,288,113,0.0,10,1011,8,61,20,71,71
1840270,2017,9,4,9,25,1200.0,0.0,1205,0.0,6,88,113,0.0,10,1017,8,68,8,83,60
1840271,2017,9,9,4,25,1143.0,0.0,1141,2.0,8,343,113,0.0,10,1012,19,74,10,82,77
1840272,2017,9,7,6,25,2157.0,0.0,2159,0.0,13,301,113,0.0,10,1011,1,27,19,64,24


In [42]:
y = df[['ArrDel15', 'ArrDelayMinutes']]

In [43]:
y.head()

,ArrDel15,ArrDelayMinutes
0,0.0,8.0
1,0.0,11.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


CLASSIFIER PIPELINE BUILDING

In [45]:
from sklearn.metrics import accuracy_score,  mean_absolute_error, classification_report,  r2_score, mean_absolute_error, mean_squared_error

In [49]:
y_pred = classifier.predict(X)
print("Accuracy of Classification: ", accuracy_score(y_pred, y['ArrDel15']))
print(classification_report(y_pred, y['ArrDel15']))
df['y_pred'] = y_pred


Accuracy of Classification:  0.975123269687014
              precision    recall  f1-score   support

         0.0       0.99      0.98      0.98   1478153
         1.0       0.91      0.97      0.94    362121

    accuracy                           0.98   1840274
   macro avg       0.95      0.97      0.96   1840274
weighted avg       0.98      0.98      0.98   1840274



In [50]:
reg_df = df[df['y_pred'] == 1.0]
reg_df = reg_df[reg_df['ArrDelayMinutes'] > 15]
X = reg_df.drop(columns = ['ArrDel15', 'ArrDelayMinutes', 'y_pred'])
y = reg_df[['ArrDel15', 'ArrDelayMinutes']]
ypred = regressor.predict(X)


In [51]:
reg_df['ArrDelayMinutes (Predicted)'] = ypred
print(reg_df[['y_pred', 'ArrDelayMinutes (Predicted)']])
#print(classification_report(ypred, y['ArrDelayMinutes']))
reg_df

         y_pred  ArrDelayMinutes (Predicted)
9           1.0                        38.63
13          1.0                        36.06
15          1.0                        40.56
20          1.0                       337.98
28          1.0                        58.62
...         ...                          ...
1840202     1.0                        63.27
1840207     1.0                       363.61
1840212     1.0                        26.01
1840251     1.0                        25.99
1840254     1.0                        34.52

[340679 rows x 2 columns]


,Year,Month,Origin,Dest,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,ArrDel15,...,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,humidity,y_pred,ArrDelayMinutes (Predicted)
9,2016,1,13,6,10,711.0,0.0,710,1.0,1.0,...,0.0,10,1024,87,35,9,37,78,1.0,38.63
13,2016,1,13,6,14,737.0,1.0,645,52.0,1.0,...,0.1,10,1017,66,37,21,40,90,1.0,36.06
15,2016,1,13,6,17,728.0,1.0,645,43.0,1.0,...,0.4,5,1009,100,38,28,44,80,1.0,40.56
20,2016,1,13,6,22,1229.0,1.0,645,344.0,1.0,...,1.5,9,1004,100,41,32,46,85,1.0,337.98
28,2016,1,6,13,2,1929.0,1.0,1825,64.0,1.0,...,0.0,10,1015,0,29,25,38,73,1.0,58.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840202,2017,9,6,8,24,2301.0,1.0,2143,78.0,1.0,...,0.0,10,1017,21,68,12,72,85,1.0,63.27
1840207,2017,9,4,9,24,1640.0,1.0,1034,366.0,1.0,...,0.0,10,1018,0,65,11,81,60,1.0,363.61
1840212,2017,9,7,14,24,1508.0,0.0,1503,5.0,1.0,...,0.0,10,1012,0,26,10,72,18,1.0,26.01
1840251,2017,9,8,6,25,1410.0,0.0,1410,0.0,1.0,...,0.0,10,1009,7,54,19,80,42,1.0,25.99


In [24]:
print("mae: ", mean_absolute_error(ypred, y['ArrDelayMinutes']))
print("mse: ", mean_squared_error(ypred, y['ArrDelayMinutes']))
mse = mean_squared_error(ypred, y['ArrDelayMinutes'])
rmse = mse ** 0.5  
print("RMSE: ", rmse)
print("R2 score: ", r2_score(ypred, y['ArrDelayMinutes']))

mae:  6.821534779772439
mse:  117.95013464497937
RMSE:  10.860485009656768
R2 score:  0.9780703563890215


Regression Analysis

In [26]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score
reg_df['ArrDelayMinutes']

9           44.0
13          31.0
15          26.0
20         335.0
28          56.0
           ...  
1840202     60.0
1840207    354.0
1840212     19.0
1840251     25.0
1840254     43.0
Name: ArrDelayMinutes, Length: 340679, dtype: float64

In [53]:
#15 - 100
r1 = reg_df[(reg_df['ArrDelayMinutes'] >= 15) & (reg_df['ArrDelayMinutes'] <= 100)]
mae = mean_absolute_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
mse = mean_squared_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
rmse = mse ** 0.5  
r2 = r2_score(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])

print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("R-Squared: ", r2)


r1.shape

MAE:  6.067896684876581
MSE:  79.18447474527349
RMSE:  8.898565881380746
R-Squared:  0.8423267625858485


(277766, 24)

In [56]:
#100 - 200
r1 = reg_df[(reg_df['ArrDelayMinutes'] >= 100) & (reg_df['ArrDelayMinutes'] <= 200)]
mae = mean_absolute_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
mse = mean_squared_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
rmse = mse ** 0.5  
r2 = r2_score(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])

print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("R-Squared: ", r2)
r1.shape

MAE:  9.871124761259319
MSE:  262.5997933782303
RMSE:  16.204931143890438
R-Squared:  0.6474099608557018


(48693, 24)

In [58]:
#200 - 500
r1 = reg_df[(reg_df['ArrDelayMinutes'] >= 200) & (reg_df['ArrDelayMinutes'] <= 500)]
mae = mean_absolute_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
mse = mean_squared_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
rmse = mse ** 0.5  
r2 = r2_score(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])

print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("R-Squared: ", r2)
r1.shape

MAE:  11.03015571536003
MSE:  365.6478784796328
RMSE:  19.1219214118151
R-Squared:  0.9163668670632706


(14184, 24)

In [61]:
#500 - 1000
r1 = reg_df[(reg_df['ArrDelayMinutes'] >= 500) & (reg_df['ArrDelayMinutes'] <= 1000)]
mae = mean_absolute_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
mse = mean_squared_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
rmse = mse ** 0.5  
r2 = r2_score(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])

print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("R-Squared: ", r2)
r1.shape

MAE:  9.443426258992805
MSE:  253.1029004496403
RMSE:  15.909208039674391
R-Squared:  0.9872136416878091


(1112, 24)

In [62]:
#1000 - 2000
r1 = reg_df[(reg_df['ArrDelayMinutes'] >= 1000) & (reg_df['ArrDelayMinutes'] <= 2000)]
mae = mean_absolute_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
mse = mean_squared_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
rmse = mse ** 0.5  
r2 = r2_score(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])

print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("R-Squared: ", r2)
r1.shape

MAE:  12.266763005780339
MSE:  486.3557797687857
RMSE:  22.053475457822643
R-Squared:  0.9821435685262953


(173, 24)

In [64]:
r1['ArrDelayMinutes']

5335       1049.0
13660      1111.0
33969      1316.0
83463      1438.0
86723      1244.0
            ...  
1752312    1187.0
1774925    1020.0
1798679    1210.0
1800524    1045.0
1811586    1209.0
Name: ArrDelayMinutes, Length: 173, dtype: float64

In [65]:
 r1['ArrDelayMinutes (Predicted)']

5335       1053.68
13660      1106.89
33969      1316.22
83463      1435.00
86723      1268.48
            ...   
1752312    1207.56
1774925    1031.50
1798679    1175.05
1800524    1036.42
1811586    1214.23
Name: ArrDelayMinutes (Predicted), Length: 173, dtype: float64